In [217]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
from trade.helpers.helper import CustomCache
from trade.helpers.helper_types import SingletonMetaClass
from trade.optionlib.config.defaults import DIVIDEND_LOOKBACK_YEARS, DIVIDEND_FORECAST_METHOD
from trade.optionlib.config.types import DiscreteDivGrowthModel, DivType
from trade.optionlib.assets.dividend import (
    get_vectorized_dividend_scehdule,
    vector_convert_to_time_frac,
    vectorized_discrete_pv,
    get_vectorized_dividend_rate,
    get_vectorized_continuous_dividends,
    get_vectorized_dividend_rate,
    get_div_histories,
    _dual_project_dividends,
    ScheduleEntry
)
import os
from EventDriven.riskmanager.market_data import MarketTimeseries
import pandas as pd
from typing import Optional, Dict, Union, List
from typing import overload, Literal
DM_GEN_PATH = Path(os.getenv("GEN_CACHE_PATH")) / "dm_gen_cache"
TS = MarketTimeseries()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2016-12-30         NaN
2017-01-02         NaN
2017-01-03    0.005323
2017-01-04    0.005329
2017-01-05    0.005302
                ...   
2025-12-31    0.000956
2026-01-01         NaN
2026-01-02    0.000959
2026-01-05    0.000973
2026-01-06    0.000991
Length: 2353, dtype: float64

In [215]:
TS.get_at_index("AAPL", "2023-01-03").__dict__

{'sym': 'AAPL',
 'date': '2023-01-03',
 'spot': open                 128.34378
 high                128.954561
 low                 122.324586
 close               123.211212
 volume               112117500
 chain_price         123.211212
 unadjusted_close    492.844849
 split_ratio                1.0
 cum_split                  4.0
 split_factor               1.0
 max_cum_split              4.0
 is_split_date            False
 Name: 2023-01-03 00:00:00, dtype: object,
 'chain_spot': open                       128.34378
 high                      128.954561
 low                       122.324586
 close                     123.211212
 volume                     112117500
 chain_price               123.211212
 unadjusted_close        27599.311523
 split_ratio                      1.0
 cum_split                      224.0
 split_factor                     1.0
 max_cum_split                  224.0
 is_split_date                  False
 cum_split_from_start             4.0
 Name: 2023-01-03 

In [78]:
hist = get_div_histories(["aapl"])
hist

{'aapl':                     amount
 ex_dividend_date          
 1987-05-11        0.000536
 1987-08-10        0.000536
 1987-11-17        0.000714
 1988-02-12        0.000714
 1988-05-16        0.000714
 ...                    ...
 2024-11-08        0.250000
 2025-02-10        0.250000
 2025-05-12        0.260000
 2025-08-11        0.260000
 2025-11-10        0.260000
 
 [89 rows x 1 columns]}

In [145]:
_dual_project_dividends(
    valuation_date="2026-11-11",
    end_date="2026-12-31",
    div_history=hist['aapl'],
    inferred_growth_rate=0.10
)

[autoreload of trade.optionlib.assets.dividend failed: Traceback (most recent call last):
  File "/Users/chiemelienwanisobi/miniconda3/envs/openbb_new_use/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/chiemelienwanisobi/miniconda3/envs/openbb_new_use/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/chiemelienwanisobi/miniconda3/envs/openbb_new_use/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/Users/chiemelienwanisobi/miniconda3/envs/openbb_new_use/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 365, in update_class
    update_instances(old, new)
  File "/Users/chiemelienwanisobi/miniconda3/envs/openbb_new_use/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 323, in update_instances
    obj

([], [], Timestamp('2026-11-11 00:00:00'))

In [57]:
div_schedule = get_vectorized_dividend_scehdule(
    tickers = ["AAPL"],
    end_dates = ["2026-12-31"],
    start_dates=["2024-01-01"],
    method=DiscreteDivGrowthModel.CONSTANT_AVG.value,
)

div_schedule[0].schedule

[(datetime.date(2024, 2, 9), 0.24), (datetime.date(2024, 5, 10), 0.25), (datetime.date(2024, 8, 12), 0.25), (datetime.date(2024, 11, 8), 0.25), (datetime.date(2025, 2, 10), 0.25), (datetime.date(2025, 5, 12), 0.26), (datetime.date(2025, 8, 11), 0.26), (datetime.date(2025, 11, 10), 0.26), (datetime.date(2026, 2, 10), np.float64(0.26)), (datetime.date(2026, 5, 10), np.float64(0.26285714285714284)), (datetime.date(2026, 8, 10), np.float64(0.26574568288854006)), (datetime.date(2026, 11, 10), np.float64(0.26866596511808444))]


[autoreload of trade.optionlib.assets.dividend failed: Traceback (most recent call last):
  File "/Users/chiemelienwanisobi/miniconda3/envs/openbb_new_use/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/chiemelienwanisobi/miniconda3/envs/openbb_new_use/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/chiemelienwanisobi/miniconda3/envs/openbb_new_use/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/Users/chiemelienwanisobi/miniconda3/envs/openbb_new_use/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 365, in update_class
    update_instances(old, new)
  File "/Users/chiemelienwanisobi/miniconda3/envs/openbb_new_use/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 323, in update_instances
    obj

[<ScheduleEntry: 2024-02-09 - 0.24>,
 <ScheduleEntry: 2024-05-10 - 0.25>,
 <ScheduleEntry: 2024-08-12 - 0.25>,
 <ScheduleEntry: 2024-11-08 - 0.25>,
 <ScheduleEntry: 2025-02-10 - 0.25>,
 <ScheduleEntry: 2025-05-12 - 0.26>,
 <ScheduleEntry: 2025-08-11 - 0.26>,
 <ScheduleEntry: 2025-11-10 - 0.26>,
 <ScheduleEntry: 2026-02-10 - 0.26>,
 <ScheduleEntry: 2026-05-10 - 0.26285714285714284>,
 <ScheduleEntry: 2026-08-10 - 0.26574568288854006>,
 <ScheduleEntry: 2026-11-10 - 0.26866596511808444>]

In [208]:
get_vectorized_dividend_rate(
    t
)

Signature:
get_vectorized_dividend_rate(
    tickers: Union[str, List[str]],
    spots: List[float],
    valuation_dates: List[float],
)
Docstring:
Get the vectorized dividend rate for a list of tickers based on their historical dividend data.

tickers: str or List[str] - Ticker symbols of the stocks.
spots: List[float] - Current spot prices for each ticker.
valuation_dates: List[datetime] - Dates for which to calculate the dividend rates.

Returns a numpy array of dividend rates.
File:      ~/cloned_repos/QuantTools/trade/optionlib/assets/dividend.py
Type:      function

In [16]:
frac = vector_convert_to_time_frac(schedules=div_schedule, valuation_dates=["2024-01-01"], end_dates=["2026-12-31"])
# pv = vectorized_discrete_pv(
#     schedules=div_schedule,
#     r = [0.05] * len(div_schedule),
#     _valuation_dates=["2024-01-01"],
#     _end_dates=["2026-12-31"]
# )
# pv

frac

[<Schedule: 12 dividends>]

In [17]:
frac[0].schedule

[(0.10677618069815195, 0.24),
 (0.35592060232717315, 0.25),
 (0.6132785763175906, 0.25),
 (0.8542094455852156, 0.25),
 (1.111567419575633, 0.25),
 (1.3607118412046544, 0.26),
 (1.6098562628336757, 0.26),
 (1.8590006844626967, 0.26),
 (2.11088295687885, np.float64(0.26)),
 (2.3545516769336072, np.float64(0.26285714285714284)),
 (2.6064339493497606, np.float64(0.26574568288854006)),
 (2.858316221765914, np.float64(0.26866596511808444))]

In [18]:
## Cache Key construction
from dataclasses import dataclass
from datetime import datetime, date, time, timezone
from enum import Enum
from hashlib import sha1
from typing import Any, Dict, Mapping, Optional, Tuple


class Interval(str, Enum):
    INTRADAY = "intraday"  # historical intraday timestamp
    EOD = "eod"  # end-of-day daily snapshot
    NA = "na"  # not applicable

class SeriesId(str, Enum):
    HIST = "hist"
    AT_TIME = "at_time"
    SNAPSHOT = "snapshot"

In [19]:
class ArtifactType(str, Enum):
    # Market / inputs
    SPOT = "spot"
    CHAIN = "chain"
    RATES = "rates"
    DIVS = "divs"
    FWD = "forward"

    # Volatility
    IV = "iv"
    TVAR = "tvar"

    # Greeks
    GREEKS = "greeks"
    DELTA = "delta"
    GAMMA = "gamma"
    VEGA = "vega"
    THETA = "theta"
    VOMMA = "vomma"
    VANNA = "vanna"


def _norm_str(x: str) -> str:
    return x.strip().upper()

def _safe_part(x: Optional[str]) -> str:
    return x if x not in (None, "", "None") else "-"

In [20]:
def _format_value(v: Any) -> str:
    """
    Keep it simple + deterministic.
    """
    if v is None:
        return "-"
    if isinstance(v, Enum):
        return str(v.value)
    if isinstance(v, str):
        return _norm_str(v)
    if isinstance(v, bool):
        return "1" if v else "0"
    if isinstance(v, (int,)):
        return str(v)
    if isinstance(v, float):
        # avoid 0.30000000000004 style keys
        return f"{v:.12g}"
    if isinstance(v, datetime):
        # stable, compact. (no tz handling by design here)
        return v.strftime("%Y%m%dT%H%M%S")
    if isinstance(v, date):
        return v.strftime("%Y%m%d")
    
    if isinstance(v, time):
        return v.strftime("%H%M%S")
    return str(v)


def construct_cache_key(
    symbol: str,
    interval: Optional[Interval],
    artifact_type: ArtifactType,
    series_id: SeriesId,
    **extra_parts: Any,
) -> str:
    
    if series_id in (SeriesId.AT_TIME, SeriesId.SNAPSHOT):
        assert 'time' in extra_parts, "time must be provided for at_time or snapshot series_id"
        assert 'date' in extra_parts, "date must be provided for at_time or snapshot series_id"
        assert isinstance(extra_parts['time'], time), "time must be a time object"
        assert isinstance(extra_parts['date'], date), "date must be a date object"


    parts = [
        f"symbol:{_norm_str(symbol)}",
        f"interval:{_format_value(interval)}",
        f"artifact_type:{artifact_type.value}",
        f"series_id:{series_id.value}"
    ]

    for k in sorted(extra_parts.keys()):
        parts.append(f"{k}:{_format_value(extra_parts[k])}")

    return "|".join(parts)


k = construct_cache_key(
    "AAPL",
    Interval.EOD,
    ArtifactType.IV,
    SeriesId.HIST,
    date=date(2024, 1, 1),
    model="SABR",
)
print(k)


symbol:AAPL|interval:eod|artifact_type:iv|series_id:hist|date:20240101|model:SABR


In [21]:
def _parse_cache_key(key: str) -> Dict[str, str]:
    parts = key.split("|")
    result = {}
    for part in parts:
        k, v = part.split(":", 1)
        result[k] = v
    return result

print(_parse_cache_key(k))

{'symbol': 'AAPL', 'interval': 'eod', 'artifact_type': 'iv', 'series_id': 'hist', 'date': '20240101', 'model': 'SABR'}


## Building DataManagers


In [22]:
DM_GEN_PATH.as_posix(), DM_GEN_PATH

('/Users/chiemelienwanisobi/cloned_repos/QuantTools/.cache/dm_gen_cache',
 PosixPath('/Users/chiemelienwanisobi/cloned_repos/QuantTools/.cache/dm_gen_cache'))

In [165]:


from abc import ABC
from dataclasses import dataclass
from typing import Any, Callable, ClassVar, Dict, Optional, Type, TypeVar

# Assumes you already have these (from your cache_key module)
# from cache_key import construct_cache_key, Interval, ArtifactType, SeriesId

T = TypeVar("T")


@dataclass(frozen=True, slots=True)
class CacheSpec:
    """
    Optional: a small config object you can pass around, so all managers
    initialize their caches in a consistent way.

    If you already have a cache registry/factory, you may not need this.

    args:
        base_dir (Optional[Path]): Directory for cache storage.
        default_expire_days (Optional[int]): Default expiration time in days. This is how many days till the entire cache entry expires.
        default_expire_seconds (Optional[int]): Default expiration time in seconds. This is how many seconds till a single cache entry expires.
        cache_fname (Optional[str]): Foldername for the cache storage.
        clear_on_exit (bool): If True, clears the cache on exit.
    """

    base_dir: Optional[Path] = DM_GEN_PATH.as_posix()
    default_expire_days: Optional[int] = 500
    default_expire_seconds: Optional[int] = None
    cache_fname: Optional[str] = None
    clear_on_exit: bool = False


class BaseDataManager(ABC):
    """
    Foundation class for all DataManagers.

    Goals:
    - Every inheritor gets a cache.
    - Every inheritor MUST define CACHE_NAME.
    - Provide consistent key creation (namespaced).
    - Provide thin get/set/get_or_compute wrappers.
    - Keep business logic out of the base.
    """

    # --- REQUIRED by inheritors ---
    CACHE_NAME: ClassVar[str] = ""

    # --- Optional defaults for common patterns ---
    DEFAULT_INTERVAL: ClassVar[Optional["Interval"]] = None
    DEFAULT_SERIES_ID: ClassVar["SeriesId"]  # prefer explicit in subclasses

    # Internal registry to prevent accidental duplicate cache names
    _CACHE_NAME_REGISTRY: ClassVar[Dict[str, Type["BaseDataManager"]]] = {}

    def __init_subclass__(cls, **kwargs: Any) -> None:
        super().__init_subclass__(**kwargs)

        # Skip enforcement for the abstract base itself
        if cls is BaseDataManager:
            return

        cache_name = getattr(cls, "CACHE_NAME", None)

        if not isinstance(cache_name, str) or not cache_name.strip():
            raise TypeError(f"{cls.__name__} must define a non-empty class variable CACHE_NAME: str")

        cache_name = cache_name.strip()

        # Enforce uniqueness to avoid collisions
        existing = cls._CACHE_NAME_REGISTRY.get(cache_name)
        # if existing is not None and existing is not cls:
        #     raise TypeError(
        #         f"Duplicate CACHE_NAME='{cache_name}'. "
        #         f"Already used by {existing.__name__}. "
        #         f"Pick a unique CACHE_NAME for {cls.__name__}."
        #     )

        cls._CACHE_NAME_REGISTRY[cache_name] = cls

        # Optional: enforce that DEFAULT_SERIES_ID exists (if you want)
        if not hasattr(cls, "DEFAULT_SERIES_ID"):
            raise TypeError(f"{cls.__name__} must define DEFAULT_SERIES_ID (e.g., SeriesId.HIST).")

    def __init__(
        self,
        *,
        cache_spec: Optional[CacheSpec] = None,
        enable_namespacing: bool = False,
    ) -> None:
        """
        Parameters
        ----------
        cache:
            Your existing CustomCache instance (diskcache-backed).
        cache_spec:
            Optional shared configuration (base_dir, TTL defaults, etc.).
        enable_namespacing:
            If True, keys are prefixed with CACHE_NAME to avoid collisions.
        """
        self.cache_spec = cache_spec or CacheSpec(cache_fname=self.CACHE_NAME)
        self.cache = CustomCache(location=self.cache_spec.base_dir, 
                                 fname=self.cache_spec.cache_fname, 
                                 expire_days=self.cache_spec.default_expire_days,
                                 clear_on_exit=self.cache_spec.clear_on_exit)
        self.enable_namespacing = enable_namespacing

    # Key construction
    def make_key(
        self,
        *,
        symbol: str,
        interval: Optional[Interval] = None,
        artifact_type: ArtifactType,
        series_id: Optional[SeriesId] = None,
        **extra_parts: Any,
    ) -> str:
        """
        Namespaced key builder that wraps your construct_cache_key.

        You decided:
        - no caching SNAPSHOT series_id (but you might still request it)
        - time is explicit if you do AT_TIME
        """
        interval = interval if interval is not None else self.DEFAULT_INTERVAL
        series_id = series_id if series_id is not None else self.DEFAULT_SERIES_ID

        raw = construct_cache_key(
            symbol=symbol,
            interval=interval,
            artifact_type=artifact_type,
            series_id=series_id,
            **extra_parts,
        )

        if not self.enable_namespacing:
            return raw

        return f"{self.CACHE_NAME}|{raw}"

    # Cache IO
    def get(self, key: str, default: Any = None) -> Any:
        return self.cache.get(key, default=default)

    def set(self, key: str, value: Any, *, expire: Optional[int] = None) -> None:
        if expire is None:
            expire = self.cache_spec.default_expire_seconds
        self.cache.set(key, value, expire=expire)

    def delete(self, key: str) -> None:
        self.cache.delete(key)

    def contains(self, key: str) -> bool:
        return key in self.cache
    
    def cache_it(self, key: str, value: Any, *, expire: Optional[int] = None) -> None:
        raise NotImplementedError(f"{self.__class__.__name__}.cache() not implemented.")

    def get_or_compute(
        self,
        key: str,
        compute_fn: Callable[[], T],
        *,
        expire: Optional[int] = None,
        force: bool = False,
    ) -> T:
        """
        Read-through caching helper.

        force=True bypasses cache read, recomputes and overwrites cache.
        """
        if not force:
            hit = self.cache.get(key, default=None)
            if hit is not None:
                return hit  # type: ignore[return-value]

        value = compute_fn()
        self.set(key, value, expire=expire)
        return value

    # Offload hook (cron calls this)
    def offload(self, *args: Any, **kwargs: Any) -> None:
        """
        Optional standard hook.

        You can override in subclasses or implement a shared offloader that
        knows how to iterate keys / export values. Keeping it as a stub here
        avoids forcing a storage design too early.
        """
        raise NotImplementedError(f"{self.__class__.__name__}.offload() not implemented.")



### Dividends DataManager

In [166]:

@dataclass
class DividendsConfig(metaclass=SingletonMetaClass):
    default_lookback_years: int = DIVIDEND_LOOKBACK_YEARS
    default_forecast_method: DiscreteDivGrowthModel = DiscreteDivGrowthModel.CONSTANT
    dividend_type: DivType = DivType.DISCRETE
    include_special_dividends: bool = False

    def assert_valid(self) -> None:
        assert self.default_lookback_years > 0, "Lookback years must be positive."
        assert self.default_lookback_years <= 5, "Lookback years seems too large. Max 5."
        assert isinstance(self.default_forecast_method, DiscreteDivGrowthModel), "Invalid forecast method. Expected DiscreteDivGrowthModel Enum."
        assert isinstance(self.dividend_type, DivType), "Invalid dividend type. Expected DivType Enum."
        assert isinstance(self.include_special_dividends, bool), "include_special_dividends must be a boolean."




    def __post_init__(self) -> None:
        self.assert_valid()

    def __setattr__(self, name, value):
        super().__setattr__(name, value)
        self.assert_valid()


In [168]:
set([(1,2), (2,3), (1,2)])

{(1, 2), (2, 3)}

In [169]:
## How dividends timeseries will work:
## If discrete:
    ## All constant(+...)  will cache up to < today
    ## All None Constant will not cache
## If continuous:
    ## Rely on MarktetTimeseries to provide continuous dividend yield history. It already caches.


    

In [234]:
class DividendDataManager(BaseDataManager):
    CACHE_NAME: ClassVar[str] = "dividend_data_manager"
    DEFAULT_SERIES_ID: ClassVar["SeriesId"] = SeriesId.HIST
    CONFIG = DividendsConfig()
    
    def __init__(self, symbol: str, *, cache_spec: Optional[CacheSpec] = None, enable_namespacing: bool = False) -> None:
        super().__init__(cache_spec=cache_spec, enable_namespacing=enable_namespacing)
        self.symbol = symbol

    def cache_it(self, key: str, value: Any, *, expire: Optional[int] = None, _type: str = "discrete") -> None:

        
        ## If discrete dividends, we first check if key exists
        ## If it does, we add to it. Only values <= today.
        ## If it does not, we create new entry
        if _type == "discrete":
            existing = self.get(key, default=None)
            allowed = [entry for entry in value if entry.date <= datetime.now().date()]
            if existing is not None:
                # Merge existing and new values. We're expecting lists of ScheduleEntry
                merged = existing + allowed
                
                ## Unique by date
                uniques = list({entry.date: entry for entry in merged}.values())
                self.set(key, uniques, expire=expire)
                return
            else:
                self.set(key, allowed, expire=expire)
                return

        # For other types or if no existing, just setattr
        self.set(key, value, expire=expire)

    @overload
    def get_data(self, 
                start_date: Union[datetime, str],
                end_date: Union[datetime, str],
                div_type: Optional[DivType] = None,
                return_key: Literal[True] = True
                ) -> tuple[pd.Series | List[ScheduleEntry], str]: ...

    @overload
    def get_data(self, 
                start_date: Union[datetime, str],
                end_date: Union[datetime, str],
                div_type: Optional[DivType] = None,
                return_key: Literal[False] = False
                ) -> pd.Series | List[ScheduleEntry]: ...

    def get_data(self, 
                 start_date: Union[datetime, str],
                 end_date: Union[datetime, str],
                 div_type: Optional[DivType] = None,
                 return_key: bool = False
                 ) -> pd.Series | List[ScheduleEntry] | tuple[pd.Series | List[ScheduleEntry], str]:
        """
        Get dividend data for the symbol. Ensures internal caching logic is respected.

        Parameters
        ----------
        start_date : Union[datetime, str]
            Start date for dividend data retrieval.
        end_date : Union[datetime, str]
            End date for dividend data retrieval.
        Returns
        -------
        pd.Series | List[ScheduleEntry]
            Dividend data in the appropriate format.
        """
        div_type = DivType(div_type) if div_type is not None else self.CONFIG.dividend_type
        if div_type == DivType.DISCRETE:
            data, key = self.get_discrete_dividend_schedule(
                ticker=self.symbol,
                start_date=start_date.strftime("%Y-%m-%d") if isinstance(start_date, datetime) else start_date,
                end_date=end_date.strftime("%Y-%m-%d") if isinstance(end_date, datetime) else end_date,
            )
        elif div_type == DivType.CONTINUOUS:
             data = self.get_div_yield_history(self.symbol)
             key = None        

        else:
            raise ValueError(f"Unsupported dividend type: {div_type}")
        
        if return_key:
            return data, key
        return data 

        

    def get_discrete_dividend_schedule(
        self,
        *,
        ticker: str,
        end_date: str,
        start_date: str,
        valuation_date: Optional[str] = None,
    ):


        method = self.CONFIG.default_forecast_method.value
        lookback_years = self.CONFIG.default_lookback_years
        key = self.make_key(
            symbol=ticker,
            artifact_type=ArtifactType.DIVS,
            series_id=SeriesId.HIST,
            method=method,
            lookback_years=lookback_years,
            current_state="schedule",
            interval=Interval.NA,
        )

        available_schedule = self.get(key, default=None)
        if available_schedule is not None:
            
            ## If max date in available schedule >= end_date, we can use cache
            max_cached_date = max(entry.date for entry in available_schedule)
            min_cached_date = min(entry.date for entry in available_schedule)
            fully_covered = (min_cached_date <= datetime.strptime(start_date, "%Y-%m-%d").date()) and (max_cached_date >= datetime.strptime(end_date, "%Y-%m-%d").date())
            if fully_covered:
                print("Cache fully covers requested date range.")
                
                ## Filter to requested date range
                filtered_schedule = [entry for entry in available_schedule if start_date <= entry.date.strftime("%Y-%m-%d") <= end_date]
                return filtered_schedule, key
            else:
                print("Cache partially covers requested date range. Fetching missing data.")
        
        schedule = get_vectorized_dividend_scehdule(
            tickers=[ticker],
            end_dates=[end_date],
            start_dates=[start_date],
            method=method,
            lookback_years=lookback_years,
            valuation_dates=[valuation_date] if valuation_date else None,
        )
        raw_schedule = schedule[0].schedule
        self.cache_it(key, raw_schedule, _type="discrete")
        
        return raw_schedule, key
    
    def get_div_yield_history(self, symbol: str) -> pd.Series:
        div_history = TS.get_timeseries(symbol)
        return div_history.dividend_yield

    def offload(self, *args: Any, **kwargs: Any) -> None:
        """
        Example implementation of offload for DividendDataManager.
        """
        print(f"No offload logic implemented for {self.CACHE_NAME}")

test = DividendDataManager(symbol="AAPL")


In [241]:
test.CONFIG.dividend_type = DivType.DISCRETE
test.get_data(
    start_date="2012-10-08",
    end_date="2025-10-31",
)

Cache fully covers requested date range.


[<ScheduleEntry: 2019-02-08 - 0.1825>,
 <ScheduleEntry: 2025-02-10 - 0.25>,
 <ScheduleEntry: 2023-05-12 - 0.24>,
 <ScheduleEntry: 2017-11-10 - 0.1575>,
 <ScheduleEntry: 2015-11-05 - 0.13>,
 <ScheduleEntry: 2017-08-10 - 0.1575>,
 <ScheduleEntry: 2019-11-07 - 0.1925>,
 <ScheduleEntry: 2022-05-06 - 0.23>,
 <ScheduleEntry: 2018-11-08 - 0.1825>,
 <ScheduleEntry: 2021-05-07 - 0.22>,
 <ScheduleEntry: 2018-02-09 - 0.1575>,
 <ScheduleEntry: 2023-08-11 - 0.24>,
 <ScheduleEntry: 2024-05-10 - 0.25>,
 <ScheduleEntry: 2015-08-06 - 0.13>,
 <ScheduleEntry: 2017-02-09 - 0.1425>,
 <ScheduleEntry: 2013-08-08 - 0.108929>,
 <ScheduleEntry: 2021-08-06 - 0.22>,
 <ScheduleEntry: 2018-05-11 - 0.1825>,
 <ScheduleEntry: 2014-05-08 - 0.1175>,
 <ScheduleEntry: 2016-05-05 - 0.1425>,
 <ScheduleEntry: 2021-02-05 - 0.205>,
 <ScheduleEntry: 2018-08-10 - 0.1825>,
 <ScheduleEntry: 2016-11-03 - 0.1425>,
 <ScheduleEntry: 2013-11-06 - 0.108929>,
 <ScheduleEntry: 2020-11-06 - 0.205>,
 <ScheduleEntry: 2015-02-05 - 0.1175>,
 <